### 筆記
* 測試集: case_00210 ~ case_00299

### 讀取函式庫

In [ ]:
### 讀取函式庫 ###
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import time

### 解決 TensorFlow 2.0 程式出現 cuDNN failed to initialize 錯誤問題

In [ ]:
import tensorflow as tf
def solve_cudnn_error():
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)

solve_cudnn_error()

### 超參數設定

In [ ]:
EPOCHS = 100
INPUT_SIZE = 512
IMAGE_ORDERING = 'channels_last'
CLASS_NUM = 2

### 建立模型

In [4]:
import keras
from keras.models import *
from keras.layers import *
from keras import layers

Using TensorFlow backend.


#### Eecoder - EfficientNetB4

In [5]:
import efficientnet.keras as efn 

model_ = efn.EfficientNetB4(
    input_shape=(INPUT_SIZE,INPUT_SIZE,3), 
    weights='noisy-student'
)

model_.summary()

Model: "efficientnet-b4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 256, 256, 48) 1296        input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 256, 256, 48) 192         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 256, 256, 48) 0           stem_bn[0][0]                    
____________________________________________________________________________________

In [6]:
### B4 ###
img_input = model_.get_layer('input_1').output # (None, 512, 512, 3)
f1 = model_.get_layer('block1b_add').output # (None, 256, 256, 24)
f2 = model_.get_layer('block2d_add').output # (None, 128, 128, 32)
f3 = model_.get_layer('block3d_add').output # (None, 64, 64, 56)
f4 = model_.get_layer('block4f_add').output # (None, 32, 32, 112)
f5 = model_.get_layer('block5f_add').output # (None, 32, 32, 160)
f6 = model_.get_layer('block6h_add').output # (None, 16, 16, 272)
f7 = model_.get_layer('block7b_add').output # (None, 16, 16, 448)

#### Decoder

In [7]:
if IMAGE_ORDERING == 'channels_first':
    MERGE_AXIS = 1
elif IMAGE_ORDERING == 'channels_last':
    MERGE_AXIS = -1

o = f7

#o = (UpSampling2D((2, 2), data_format=IMAGE_ORDERING))(o)
o = (concatenate([o, f6], axis=MERGE_AXIS))
o = (ZeroPadding2D((1, 1), data_format=IMAGE_ORDERING))(o)
o = (Conv2D(512, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
o = (BatchNormalization())(o)

o = (UpSampling2D((2, 2), data_format=IMAGE_ORDERING))(o)
o = (concatenate([o, f5], axis=MERGE_AXIS))
o = (ZeroPadding2D((1, 1), data_format=IMAGE_ORDERING))(o)
o = (Conv2D(256, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
o = (BatchNormalization())(o)

#o = (UpSampling2D((2, 2), data_format=IMAGE_ORDERING))(o)
o = (concatenate([o, f4], axis=MERGE_AXIS))
o = (ZeroPadding2D((1, 1), data_format=IMAGE_ORDERING))(o)
o = (Conv2D(128, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
o = (BatchNormalization())(o)

o = (UpSampling2D((2, 2), data_format=IMAGE_ORDERING))(o)
o = (concatenate([o, f3], axis=MERGE_AXIS))
o = (ZeroPadding2D((1, 1), data_format=IMAGE_ORDERING))(o)
o = (Conv2D(64, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
o = (BatchNormalization())(o)

o = (UpSampling2D((2, 2), data_format=IMAGE_ORDERING))(o)
o = (concatenate([o, f2], axis=MERGE_AXIS))
o = (ZeroPadding2D((1, 1), data_format=IMAGE_ORDERING))(o)
o = (Conv2D(64, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
o = (BatchNormalization())(o)

o = (UpSampling2D((2, 2), data_format=IMAGE_ORDERING))(o)
o = (concatenate([o, f1], axis=MERGE_AXIS))
o = (ZeroPadding2D((1, 1), data_format=IMAGE_ORDERING))(o)
o = (Conv2D(32, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
o = (BatchNormalization())(o)

o = (UpSampling2D((2, 2), data_format=IMAGE_ORDERING))(o)
o = (ZeroPadding2D((1, 1), data_format=IMAGE_ORDERING))(o)
o = (Conv2D(32, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
o = (BatchNormalization())(o)

o = Conv2D(CLASS_NUM, (3, 3), padding='same', data_format=IMAGE_ORDERING)(o) # 輸出層

### 模仿 keras_segmentation 在輸出層之後額外添加這幾層 ###
if IMAGE_ORDERING == 'channels_first':
    o = (Reshape((CLASS_NUM, INPUT_SIZE * INPUT_SIZE)))(o)
    o = (Permute((2, 1)))(o)
elif IMAGE_ORDERING == 'channels_last':
    o = (Reshape((INPUT_SIZE * INPUT_SIZE, CLASS_NUM)))(o)
o = (Activation('softmax'))(o)
###

In [8]:
model = Model(img_input, o)

#### 查看模型架構

B4：
* Total params: 22,355,674
* Trainable params: 22,231,882
* Non-trainable params: 123,792

In [9]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 256, 256, 48) 1296        input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 256, 256, 48) 192         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 256, 256, 48) 0           stem_bn[0][0]                    
____________________________________________________________________________________________

### KiTS19測試集 - 腎臟區域預測

In [10]:
dataset_dir = 'DATA_0919\\WW600WL100'
x_test_path = 'DATA_0919\\WW600WL100\\test\\images'
checkpoints_path = 'DATA_0919\WW600WL100\\Model_EfficientNetB4-Unet_0919'
model_name = 'EfficientNetB4-Unet'

**載入腎臟區域預測模型的模型參數**

In [11]:
### 載入模型參數 ###
model.load_weights(os.path.join(checkpoints_path, model_name + '_' + str(EPOCHS) + '.h5'))

**生成預測結果**

In [12]:
from evaluate import *
preds = predict_from_folder(model, x_test_path, INPUT_SIZE, 2)

100%|████████████████████████████████████████████████████████████████████████████| 19740/19740 [17:38<00:00, 18.65it/s]


**輸出模型的預測結果和模型從原始CT影像中切割出來的腎臟區域**

In [13]:
from evaluate import *

st = time.time()

_ = evaluate_model(
    image_dir = x_test_path, 
    label_dir = None,
    checkpoints_path = None,
    calculate_predicting_indicators = False,
    output_predicted_result = True, 
    segment_out_predicted_region_from_original_images = True, 
    roi_description = 'tumor as kidney', 
    preds = preds,
    batch_process_slice_point = None)

ed = time.time()
spend_time = ed - st
print('花費時間(秒)：' + str(spend_time))

----------預測結果資料型態轉換(第1批資料)----------
-----建立新資料夾：DATA_0919\WW600WL100\test\annotations_tumor as kidney_predicted-----
---------開始輸出模型預測結果(第1批資料)----------
目前進度：第500張照片
目前進度：第1000張照片
目前進度：第1500張照片
目前進度：第2000張照片
目前進度：第2500張照片
目前進度：第3000張照片
目前進度：第3500張照片
目前進度：第4000張照片
目前進度：第4500張照片
目前進度：第5000張照片
目前進度：第5500張照片
目前進度：第6000張照片
目前進度：第6500張照片
目前進度：第7000張照片
目前進度：第7500張照片
目前進度：第8000張照片
目前進度：第8500張照片
目前進度：第9000張照片
目前進度：第9500張照片
目前進度：第10000張照片
目前進度：第10500張照片
目前進度：第11000張照片
目前進度：第11500張照片
目前進度：第12000張照片
目前進度：第12500張照片
目前進度：第13000張照片
目前進度：第13500張照片
目前進度：第14000張照片
目前進度：第14500張照片
目前進度：第15000張照片
目前進度：第15500張照片
目前進度：第16000張照片
目前進度：第16500張照片
目前進度：第17000張照片
目前進度：第17500張照片
目前進度：第18000張照片
目前進度：第18500張照片
目前進度：第19000張照片
目前進度：第19500張照片
-----建立新資料夾：DATA_0919\WW600WL100\test\images_only_containing_predicted_roi_tumor as kidney-----
----------開始生成並輸出只包含模型預測區域的圖片(第1批資料)----------
目前進度：第500張照片
目前進度：第1000張照片
目前進度：第1500張照片
目前進度：第2000張照片
目前進度：第2500張照片
目前進度：第3000張照片
目前進度：第3500張照片
目前進度：第4000張照片
目前進度：第4500張照片
目前進度：第5000張

### KiTS19測試集 - 腎臟腫瘤區域預測

In [15]:
dataset_dir = 'DATA_0919\\WW600WL100'
x_test_path = 'DATA_0919\\WW600WL100\\test\\images_only_containing_predicted_roi_tumor as kidney'
checkpoints_path = 'DATA_0919\WW600WL100\\Model_phase2_EfficientNetB4-Unet_0920'
model_name = 'EfficientNetB4-Unet'

**載入腎臟腫瘤區域預測模型的模型參數**

In [17]:
### 載入模型參數 ###
model.load_weights(os.path.join(checkpoints_path, model_name + '_' + str(EPOCHS) + '.h5'))

**生成預測結果**

In [18]:
from evaluate import *
preds = predict_from_folder(model, x_test_path, INPUT_SIZE, 2)

100%|████████████████████████████████████████████████████████████████████████████| 19740/19740 [17:38<00:00, 18.65it/s]


**輸出模型的預測結果**

In [19]:
from evaluate import *

st = time.time()

_ = evaluate_model(
    image_dir = x_test_path, 
    label_dir = None,
    checkpoints_path = None,
    calculate_predicting_indicators = False,
    output_predicted_result = True, 
    segment_out_predicted_region_from_original_images = False, 
    roi_description = 'tumor only(tumor as 1)', 
    preds = preds,
    batch_process_slice_point = None)

ed = time.time()
spend_time = ed - st
print('花費時間(秒)：' + str(spend_time))

----------預測結果資料型態轉換(第1批資料)----------
-----建立新資料夾：DATA_0919\WW600WL100\test\annotations_tumor only(tumor as 1)_predicted-----
---------開始輸出模型預測結果(第1批資料)----------
目前進度：第500張照片
目前進度：第1000張照片
目前進度：第1500張照片
目前進度：第2000張照片
目前進度：第2500張照片
目前進度：第3000張照片
目前進度：第3500張照片
目前進度：第4000張照片
目前進度：第4500張照片
目前進度：第5000張照片
目前進度：第5500張照片
目前進度：第6000張照片
目前進度：第6500張照片
目前進度：第7000張照片
目前進度：第7500張照片
目前進度：第8000張照片
目前進度：第8500張照片
目前進度：第9000張照片
目前進度：第9500張照片
目前進度：第10000張照片
目前進度：第10500張照片
目前進度：第11000張照片
目前進度：第11500張照片
目前進度：第12000張照片
目前進度：第12500張照片
目前進度：第13000張照片
目前進度：第13500張照片
目前進度：第14000張照片
目前進度：第14500張照片
目前進度：第15000張照片
目前進度：第15500張照片
目前進度：第16000張照片
目前進度：第16500張照片
目前進度：第17000張照片
目前進度：第17500張照片
目前進度：第18000張照片
目前進度：第18500張照片
目前進度：第19000張照片
目前進度：第19500張照片
total case number: 19740
花費時間(秒)：1604.4806952476501


### 合併KiTS19測試集腎臟和腎臟腫瘤區域的預測結果

In [20]:
pred_kidney_path = 'DATA_0919\\WW600WL100\\test\\annotations_tumor as kidney_predicted'
pred_tumor_path = 'DATA_0919\\WW600WL100\\test\\annotations_tumor only(tumor as 1)_predicted'
pred_kidney_and_tumor_output_path = 'DATA_0919\\WW600WL100\\test\\annotations_kidney and tumor_predicted'

st = time.time()

if not os.path.exists(pred_kidney_and_tumor_output_path):
    os.makedirs(pred_kidney_and_tumor_output_path)
    print('-----建立新資料夾：' + pred_kidney_and_tumor_output_path + '-----')

for i in range(len(os.listdir(pred_kidney_path))):
    image_ori_name = os.listdir(pred_kidney_path)[i]
    pred_kidney = cv2.imread(
        os.path.join(pred_kidney_path, os.listdir(pred_kidney_path)[i]), cv2.IMREAD_GRAYSCALE)
    pred_tumor = cv2.imread(
        os.path.join(pred_tumor_path, os.listdir(pred_tumor_path)[i]), cv2.IMREAD_GRAYSCALE)
    
    pred_kidney_and_tumor = pred_kidney.copy()
    pred_kidney_and_tumor[pred_tumor == 1] = 2
    
    cv2.imwrite(os.path.join(pred_kidney_and_tumor_output_path, image_ori_name), pred_kidney_and_tumor)
    
    if (i + 1) % 500 == 0:
        print(f'目前進度：{i + 1} / {len(os.listdir(pred_kidney_path))}')
        
ed = time.time()
spend_time = ed - st
print('花費時間(秒)：' + str(spend_time))

-----建立新資料夾：DATA_0919\WW600WL100\test\annotations_kidney and tumor_predicted-----
目前進度：500 / 19740
目前進度：1000 / 19740
目前進度：1500 / 19740
目前進度：2000 / 19740
目前進度：2500 / 19740
目前進度：3000 / 19740
目前進度：3500 / 19740
目前進度：4000 / 19740
目前進度：4500 / 19740
目前進度：5000 / 19740
目前進度：5500 / 19740
目前進度：6000 / 19740
目前進度：6500 / 19740
目前進度：7000 / 19740
目前進度：7500 / 19740
目前進度：8000 / 19740
目前進度：8500 / 19740
目前進度：9000 / 19740
目前進度：9500 / 19740
目前進度：10000 / 19740
目前進度：10500 / 19740
目前進度：11000 / 19740
目前進度：11500 / 19740
目前進度：12000 / 19740
目前進度：12500 / 19740
目前進度：13000 / 19740
目前進度：13500 / 19740
目前進度：14000 / 19740
目前進度：14500 / 19740
目前進度：15000 / 19740
目前進度：15500 / 19740
目前進度：16000 / 19740
目前進度：16500 / 19740
目前進度：17000 / 19740
目前進度：17500 / 19740
目前進度：18000 / 19740
目前進度：18500 / 19740
目前進度：19000 / 19740
目前進度：19500 / 19740


### 將KiTS19測試集的預測結果轉成nii.gz的格式

In [31]:
predictions_path = 'DATA_0919\\WW600WL100\\test\\predictions_nii'
predictions_compressed_path = 'DATA_0919\\WW600WL100\\test\\predictions_nii_gzip'

st = time.time()

import gzip
import shutil
import nibabel as nib

if not os.path.exists(predictions_path):
    os.makedirs(predictions_path)
    print('-----建立新資料夾：' + predictions_path + '-----')
    
if not os.path.exists(predictions_compressed_path):
    os.makedirs(predictions_compressed_path)
    print('-----建立新資料夾：' + predictions_compressed_path + '-----')

preds_onePatient = []

# 取得測試集第一位病患的編號
separator = '_'
prev_patient_idx = separator.join(os.listdir(pred_kidney_and_tumor_output_path)[0].split('_')[:-1])[-5:]

counter = 0

print('----------開始將KiTS19測試集的預測結果轉成nii.gz的格式----------')

for i in range(len(os.listdir(pred_kidney_and_tumor_output_path))):
    
    # 取得目前處理的影像對應的病患編號
    separator = '_'
    patient_idx = separator.join(os.listdir(pred_kidney_and_tumor_output_path)[i].split('_')[:-1])[-5:]

    if patient_idx != prev_patient_idx: # 判斷是否到達下一位病患的影像

        preds_onePatient = np.asarray(preds_onePatient)
        preds_onePatient_nii = nib.Nifti1Image(preds_onePatient, affine=np.eye(4))
        nib.save(preds_onePatient_nii, os.path.join(predictions_path, 'prediction_' + prev_patient_idx +'.nii'))
        
        f_in_path = os.path.join(predictions_path, 'prediction_' + prev_patient_idx +'.nii')
        f_out_path = os.path.join(predictions_compressed_path, 'prediction_' + prev_patient_idx +'.nii.gz')
        
        with open(f_in_path, 'rb') as f_in:
            with gzip.open(f_out_path , 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)

        preds_onePatient = []

    preds_onePatient.append(
        cv2.imread(
            os.path.join(pred_kidney_and_tumor_output_path, os.listdir(pred_kidney_and_tumor_output_path)[i]), 
            cv2.IMREAD_GRAYSCALE))

    if i == len(os.listdir(pred_kidney_and_tumor_output_path)) - 1: 
        # 判斷是否為最後一張的影像
        
        preds_onePatient = np.asarray(preds_onePatient)
        preds_onePatient_nii = nib.Nifti1Image(preds_onePatient, affine=np.eye(4))
        nib.save(preds_onePatient_nii, os.path.join(predictions_path, 'prediction_' + patient_idx +'.nii'))
        
        f_in_path = os.path.join(predictions_path, 'prediction_' + patient_idx +'.nii')
        f_out_path = os.path.join(predictions_compressed_path, 'prediction_' + patient_idx +'.nii.gz')
        
        with open(f_in_path, 'rb') as f_in:
            with gzip.open(f_out_path , 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
 
    prev_patient_idx = patient_idx

    counter += 1
    if counter % 500 == 0:
        print('目前進度：第' + str(counter) + '張照片')

ed = time.time()
spend_time = ed - st
print('花費時間(秒)：' + str(spend_time))

-----建立新資料夾：DATA_0919\WW600WL100\test\predictions_nii-----
-----建立新資料夾：DATA_0919\WW600WL100\test\predictions_nii_gzip-----
----------開始將KiTS19測試集的預測結果轉成nii.gz的格式----------
目前進度：第500張照片
目前進度：第1000張照片
目前進度：第1500張照片
目前進度：第2000張照片
目前進度：第2500張照片
目前進度：第3000張照片
目前進度：第3500張照片
目前進度：第4000張照片
目前進度：第4500張照片
目前進度：第5000張照片
目前進度：第5500張照片
目前進度：第6000張照片
目前進度：第6500張照片
目前進度：第7000張照片
目前進度：第7500張照片
目前進度：第8000張照片
目前進度：第8500張照片
目前進度：第9000張照片
目前進度：第9500張照片
目前進度：第10000張照片
目前進度：第10500張照片
目前進度：第11000張照片
目前進度：第11500張照片
目前進度：第12000張照片
目前進度：第12500張照片
目前進度：第13000張照片
目前進度：第13500張照片
目前進度：第14000張照片
目前進度：第14500張照片
目前進度：第15000張照片
目前進度：第15500張照片
目前進度：第16000張照片
目前進度：第16500張照片
目前進度：第17000張照片
目前進度：第17500張照片
目前進度：第18000張照片
目前進度：第18500張照片
目前進度：第19000張照片
目前進度：第19500張照片
